<h3><center>VWAP, Sharpe and Alpha Analysis<br>
CUNY SPS MSDS<br>
Professor Samuel Gralnick<br>
Team Sugar Cane: Euclid Zhang, Jie Zou, Zhenni Xie<br>
Fall 2022</center></h3>

In this analysis, we are performing calculations of the Volume-Weighted Average Price (VWAP), Sharpe Ratio, and Alpha of our investment portfolio and the individual assets in the portfolio, based on the asset prices from 8/25/2022 to 10/7/2022

In [422]:
#Loading packages

import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
import pandas_datareader.data as pdr
import pandas as pd
import scipy.stats as stats
import math
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [423]:
#The start date and end date of the assets' prices used for the calculations
start = datetime.strptime('2022-08-25', '%Y-%m-%d')
end = datetime.strptime('2022-10-08', '%Y-%m-%d')

Our investment portfolio includes the following assets, with purchasing prices (close prices) from 9/16/2022. In this analysis, **we will not include the U.S. 30 Year Treasury bonds in our calculation** because the metrics we are calculating are created to the compare portfolios of equities.

In [424]:
#Load the infromation of the assets in our portfolio
assets = pd.read_csv("portfolio assets.csv")
assets

,Ticker,Type,Sector,Price,Units,Amount
0,US30Y,U.S. 30 Year Treasury,NaN,90.44,467317.00,42262981.19
1,HSON,Common Stock,COMMERCIAL SERVICES,34.95,36138.24,1263031.49
2,GOGO,Common Stock,COMMUNICATIONS,13.32,94822.19,1263031.57
3,JAKK,Common Stock,CONSUMER DURABLES,23.37,54044.99,1263031.42
4,CALM,Common Stock,CONSUMER NON-DURABLES,56.85,22216.91,1263031.33
5,HRB,Common Stock,CONSUMER SERVICES,44.93,28111.09,1263031.27
6,HDSN,Common Stock,DISTRIBUTION SERVICES,7.94,159071.98,1263031.52
7,BELFA,Common Stock,ELECTRONIC TECHNOLOGY,28.52,44285.81,1263031.30
8,ARLP,Common Stock,ENERGY MINERALS,23.88,52890.76,1263031.35
9,CI,Common Stock,HEALTH SERVICES,290.35,4350.03,1263031.21


In [425]:
#Calculate the weights of the assets in our portfolio
weights = np.array(assets['Amount'][1:]/sum(assets['Amount'][1:]))

We then retrieve the prices and volumes of the common stocks and ETFs in our portfolio, using the yfinance package

In [426]:
#retrieve the prices and volumes of the common stocks and ETFs in our portfolio, using the yfinance package

yf.pdr_override()

tickers = list(assets['Ticker'])[1:]

historical_data = pdr.DataReader(tickers, start, end , data_source='yahoo')
historical_data = historical_data.reset_index()
historical_data.Date = historical_data.Date.apply(datetime.date)
historical_data.set_index('Date', inplace = True)


[*********************100%***********************]  29 of 29 completed


First we calculate the Volume-Weighted Average Price (VWAP) using the following formula

<TABLE STYLE="display:inline-table; font-size:100%; border-collapse:collapse; vertical-align:middle"><TR><TD STYLE="padding:0px; border-bottom: solid 1px; width:500px; height: 20px; text-align:center"><span>&Sigma;</span> [(daily high price + daily low price + daily close price) * daily trading volume]</TD></TR> <TR><TD STYLE="padding:0px; text-align:center"><span>&Sigma;</span> daily trading volume</TD></TR></TABLE>

In [427]:
total_traded = ((historical_data['High'] + historical_data['Low'] + historical_data['Close'])/3)*historical_data['Volume']
total_traded.fillna(0, inplace = True)
VWAP = pd.DataFrame(total_traded.sum() / historical_data['Volume'].sum())
VWAP.columns = ['VWAP']

In [428]:
VWAP = VWAP.merge(right = assets[['Ticker','Price']], how = "inner", left_index = True, right_on="Ticker").set_index('Ticker')
VWAP = VWAP.merge(pd.DataFrame(historical_data['Adj Close'].iloc[historical_data['Adj Close'].shape[0]-1,]),
                 how = "inner", left_index = True, right_index = True)
VWAP.rename(columns={'Price': 'Purchased Price'}, inplace=True)

#adding VWAP for the portfolio
port_VWAP = pd.DataFrame(VWAP.T.dot(weights)).T
port_VWAP.index = ["Portfolio"]
VWAP = round(VWAP.append(port_VWAP),2)
VWAP.tail(5)

,VWAP,Purchased Price,2022-10-07
TTT,69.32,67.66,78.91
TYO,12.78,12.52,13.84
UUP,29.78,29.40,30.30
YCS,63.35,63.33,65.74
Portfolio,76.93,76.79,78.40


Next, we calculate the Sharpe Ratio using the following formula

<TABLE STYLE="display:inline-table; font-size:100%; border-collapse:collapse; vertical-align:middle"><TR><TD STYLE="padding:0px; border-bottom: solid 1px; width:500px; height: 20px; text-align:center">return of portfolio - risk-free rate</TD></TR> <TR><TD STYLE="padding:0px; text-align:center">standard deviation of the portfolio's return</TD></TR></TABLE>

For the risk-free rate, we use the yield rate of the U.S. 3 Month Treasury Bill as of 10/07/2022, 3.365% (https://www.marketwatch.com/investing/bond/tmubmusd03m?countrycode=bx). We calculated the daily return using 251 trade days in a year. The daily gain is

In [429]:
risk_free_rate = 1.03365**(1/251) -1
risk_free_rate

0.0001318661736162685

We then calculate the Sharpe Ratios using the daily Adjusted Close Price

In [430]:
close_prices = historical_data['Adj Close']
returns = close_prices.pct_change().dropna()
returns['Portfolio'] = returns.dot(weights)
sharp_ratios = pd.DataFrame((returns.mean() - risk_free_rate) / returns.std())
sharp_ratios.columns = ['Sharpe Ratio']

In [431]:
sharp_ratios.tail(5)

,Sharpe Ratio
TTT,0.258891
TYO,0.291776
UUP,0.186173
YCS,0.317407
Portfolio,0.040293


Now we calcuate the alpha of the portfolio and assets, which is derived from the following model

return of investment - risk-free rate = alpha + beta * (return of benchmark - risk-free rate) + random error

In this analysis, we calculate three different alphas using the S&P 500 ("^GSPC"), Dow Jones ("^DJI"), and Nasdaq Composite ("^IXIC") indexes as benchmarks. Instead of using the current risk-free rate, we use the historical yield rates (U.S. 3 Month Treasury Bill) within the same time frame to calculate the alphas.

In [432]:
#load the U.S. 3 Month Treasury Bill yield rates, downloaded from 
#https://www.marketwatch.com/investing/bond/tmubmusd03m?countrycode=bx

risk_free_rates = pd.read_csv("BOND_BX_XTUP_TMUBMUSD03M.csv")
risk_free_rates.Date  = pd.to_datetime(risk_free_rates.Date)
risk_free_rates.set_index('Date', inplace = True)
risk_free_rates.sort_index(inplace = True)
risk_free_rates['Close']  = risk_free_rates['Close'].str.rstrip("%").astype("float") / 100
risk_free_rates['Close'] = (1 + risk_free_rates['Close'])**(1/251)-1
risk_free_rates = risk_free_rates[str(start.date()):str(end.date())]['Close']

In [433]:
#retrieve the prices of the market indexes, using the yfinance package

tickers = tickers = ["^GSPC", "^DJI", "^IXIC"]
index_prices = pdr.DataReader(tickers, start, end , data_source='yahoo')
index_prices = index_prices['Adj Close']

index_prices = index_prices.reset_index()
index_prices.Date = index_prices.Date.apply(datetime.date)
index_prices.set_index('Date', inplace = True)

#calcuate the daily percent returns of the indexes
market_rates = index_prices.pct_change().dropna()

[*********************100%***********************]  3 of 3 completed


In [434]:
#define a function to calculate the alphas. Addtionally, we can get the beta and R-squared from the results.

#The code is from the following post
#https://firemymoneymanager.com/getting-started-using-python-to-find-alpha/
def market_compare(market_ticker):
    #new dataframe to store the results of the calculations
    regressions = pd.DataFrame(columns = ['ticker', 'alpha', 'beta', 'rsquared'])

    for i in range(0, len(returns.columns)):
        # for calculating y, we must subtract the risk free rate
        y = (returns[returns.columns[i]] - risk_free_rates).dropna()
        X = (market_rates[market_ticker] - risk_free_rates).dropna()
        X = sm.add_constant(X)
        X.columns = ['const','Mkt-RF']

        # training the model
        model = sm.OLS(y, X).fit()

        # append the results
        regressions.loc[i] = [ 
            returns.columns[i],
            model.params['const'],
            model.params['Mkt-RF'],
            model.rsquared
        ]
    regressions.set_index('ticker', inplace = True)
    return regressions

Calculate the alphas using three different market indexes

In [435]:
S_P500_Compare = market_compare('^GSPC')
Dow_Jones_Compare = market_compare('^DJI')
Nasdaq_Compare = market_compare('^IXIC')

The following shows some of the results from the calculation using S&P 500 index as the benchmark

In [436]:
S_P500_Compare.tail(5)

,alpha,beta,rsquared
ticker,,,
TTT,0.008250,-0.504184,0.042376
TYO,0.003927,-0.572935,0.178468
UUP,-0.000043,-0.291778,0.467197
YCS,0.002917,-0.284953,0.126693
Portfolio,0.002527,0.429578,0.362868


Fainlly, combine the reuslt of all calculations together so we can compare the numbers

In [437]:
summary = pd.DataFrame(S_P500_Compare['alpha']).merge(pd.DataFrame(Dow_Jones_Compare['alpha']), how = "inner", left_index = True, right_index = True)
summary = summary.merge(pd.DataFrame(Nasdaq_Compare['alpha']), how = "inner", left_index = True, right_index = True)
summary.columns = ['alpha_SP500', 'alpha_Dow_Jones', 'alpha_Nasdaq']
summary = sharp_ratios.merge(summary, how = "inner", left_index = True, right_index = True)
summary = VWAP.merge(summary, how = "inner", left_index = True, right_index = True)
summary

,VWAP,Purchased Price,2022-10-07,Sharpe Ratio,alpha_SP500,alpha_Dow_Jones,alpha_Nasdaq
ARLP,24.59,23.88,24.31,-0.037640,0.002477,0.002153,0.002812
ASC,9.72,10.00,9.53,-0.005351,0.000966,0.000789,0.001041
AZPN,228.50,226.37,253.41,0.278757,0.008350,0.008068,0.008586
BELFA,29.62,28.52,29.03,-0.105511,-0.001059,-0.001097,-0.000781
BSM,15.97,15.70,17.36,0.106568,0.005186,0.005072,0.005357
CALM,57.21,56.85,53.63,-0.031122,0.001702,0.001442,0.001986
CF,102.63,98.59,104.80,-0.093633,0.000446,0.000400,0.000516
CI,286.76,290.35,288.38,-0.022557,0.002893,0.003031,0.002774
CSL,294.12,291.30,292.45,-0.137457,0.001818,0.001723,0.001823
DIG,35.23,36.04,38.81,-0.009746,0.008873,0.008822,0.008746


## Conclusion

We purchased the portfolio with a price (76.79 per unit) lower than the VWAP (76.93 per unit) and the current value of a unit is 78.40. We are making a profit and we may consider selling some assets that the current value is higher than the VWAP. The Sharpe Ratio is positive, indicating that our equities has a better return than the risk-free investment. Some of the individual assets have a negative Sharpe Ratio, we may sell some of them and reallocate the fund to risk-free investments or other equities. The Alphas calculated using three different benchmarks are showing that our portfolio outperformed the three market indexes. Overall, our portfolio is performing well. Some minor adjustments may be good, but we can hold the assets for now as the observation is based on a few weeks only.